# NER实验代码

https://zhuanlan.zhihu.com/p/27597790  
https://taku910.github.io/crfpp/

In [2]:
import codecs

In [ ]:
'''
预处理：
1. 人名/nr需要进行合并
2. 时间/t需要进行合并
3. 中括号进行合并
4. 全角统一转换成半角
'''

def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

def sentence_process(line):
    line = line[:-2]
    state = 0
    state_key = ''
    concat_list = ''
    result_list = []

    for n, w in enumerate(line.split('  ')):
        if state == 2:
            #[香港/ns 特别/a 行政区/n]ns ==> 香港特别行政区/ns
            if ']' in w:
                result_list.append(concat_list + w.split('/')[0] + '/' + w.split(']')[-1])
                concat_list = ''
                state = 0
            else:
                concat_list += w.split('/')[0]
        elif state == 1:
            if w.split('/')[-1] == state_key:
                concat_list += w.split('/')[0]
            else:
                result_list.append(concat_list + '/' + state_key)
                result_list.append(w)
                concat_list = ''
                state = 0
        else:
            if w.startswith('['):
                concat_list = w[1:].split('/')[0]
                state = 2
            elif w.endswith('/nr') or w.endswith('/t'):
                state_key = w.split('/')[-1]
                concat_list = w.split('/')[0]
                state = 1
            else:
                result_list.append(w)
    return strQ2B('  '.join(result_list))

with codecs.open('../dataset/199801-utf8-new.txt', 'w', 'utf-8') as wf:
    with codecs.open('../dataset/199801-utf8.txt', 'r', 'utf-8') as f:
        for line in f.readlines():
            new_line = sentence_process(line)
            wf.write(new_line + '\n')


需要根据199801-utf8-new.txt中的内容，将每个字提取
根据IO tagger进行标注，crf++的输入程序

标注以下四类实体：
```
/t TIME
/nr PERSON
/ns LOCATION
/nt ORGANIZATION
```
将生成以下格式：
```
中 ORGANIZATION
共 ORGANIZATION
中 ORGANIZATION
央 ORGANIZATION
总 O
书 O
记 O
、 O
国 O
家 O
主 O
席 O
江 PERSON
泽 PERSON
民 PERSON
```

In [6]:
'''
生成训练集
'''

tagging_dict = {
    't': 'TIME',
    'nr': 'PERSON',
    'ns': 'LOCATION',
    'nt': 'ORGANIZATION'
}

def tagging_trans(tag):
    if tag in tagging_dict:
        return tagging_dict[tag]
    else:
        return 'O'

with codecs.open('../dataset/199801-utf8-new.txt', 'r', 'utf-8') as rf:
    with codecs.open('../tmp/199801.ner.tagging4crf.txt', 'w') as wf:
        for n, line in enumerate(rf.readlines()):
            for w in line.split('  ')[1:-1]:
                w_content, w_type = tuple(w.rsplit('/', 1))
                for c in w_content:
                    # wf.write(c + '\t' + w_type + '\t' + tagging_trans(w_type) + '\n')
                    # wf.write(c + '\t' + tagging_trans(w_type) + '\n')
                    wf.write(c + '\t' + w_type + '\n')
            wf.write('\n')


In [ ]:
# 训练代码
!cd /home/li/workspace/newsyard/ner_crf/
!crf_learn -f 1 -c 1.0 template ../tmp/199801.ner.tagging4crf.txt ../tmp/199801.ner.crf

In [ ]:
# 测试代码

test_line = "习近平在十九届中共中央政治局常委同中外记者见面时强调：" + \
            "新时代要有新气象更要有新作为，中国人民生活一定会一年更比一年好。"
with codecs.open('../tmp/199801.test.txt', 'w', 'utf-8') as wf:
    for c in test_line:
        wf.write(c + '\n')
    

In [8]:
!crf_test -m ../tmp/199801.ner.crf ../tmp/199801.test.txt

习	O
近	O
平	O
在	O
十	O
九	O
届	O
中	ORGANIZATION
共	ORGANIZATION
中	ORGANIZATION
央	ORGANIZATION
政	ORGANIZATION
治	ORGANIZATION
局	ORGANIZATION
常	O
委	O
同	O
中	O
外	O
记	O
者	O
见	O
面	O
时	O
强	O
调	O
：	O
新	O
时	O
代	O
要	O
有	O
新	O
气	O
象	O
更	O
要	O
有	O
新	O
作	O
为	O
，	O
中	LOCATION
国	LOCATION
人	O
民	O
生	O
活	O
一	O
定	O
会	O
一	O
年	O
更	O
比	O
一	O
年	O
好	O
。	O

